# **<center>Recognizing American Sign Language</center>**

<center><img src="https://miro.medium.com/max/696/0*vmgQDtKZthpE9Fel"/></center>

As we are aware, human interaction is affected by various health issues, and one such issue is deafness. With the advancement of Artificial Intelligence, we are witnessing the emergence of several solutions. In this notebook, our objective is to develop a Deep Learning model using the CNN Algorithm. This model is a crucial step in addressing the communication challenges caused by deafness.

### ***Note 1:***
The dataset used in this project consists of 25 labels instead of 26. This is because the letters 'J' and 'Z' require hand gestures for understanding. However, the dataset creator may have considered that the letter 'Z' has a distinctive appearance and can be easily identified. Hence, the dataset includes the letter 'Z' but excludes the letter 'J'.
### ***Note 2:***
It should be noted that this dataset contains a few incorrect labels. While the model rarely misclassifies images, the accuracy of the labeling can be confirmed by comparing it to the correct labeling mentioned below.

# **Installing the Required Libraries**

In case you have not installed the required libraries, you can do so by running the following code. If you already have the libraries installed, you can skip this step.

In [ ]:
!pip3 install tensorflow
!pip3  install numpy
!pip3  install pandas
!pip3  install seaborn
!pip3  install matplotlib
!pip3  install pytorch
!pip3  install sklearn
!pip3  install mlxtend

# **Importing Libraries**

In [ ]:
import numpy as np # linear algebra,math functions
import tensorflow as tf # tensorflow
import random # for seed value
import os # for folder functions
import string # for making alphabet labels as string
import pandas as pd # data processing
%matplotlib inline
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # data visualization
for dirname, _, filenames in os.walk('/kaggle/input'): # default
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
seed=42
os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# We are using random seeds for controlling randomness.

# **Data Preprocessing**

In [ ]:
train_data = pd.read_csv("C:/Users/xjont/OneDrive/Documentos/CETYS/6TO SEMESTRE/SISTEMAS INTELIGENTES/REPOSITORY/SI23-BongoCat/proyecto_sign_language/dataset/sign_mnist_train/sign_mnist_train.csv")
test_data = pd.read_csv("C:/Users/xjont/OneDrive/Documentos/CETYS/6TO SEMESTRE/SISTEMAS INTELIGENTES/REPOSITORY/SI23-BongoCat/proyecto_sign_language/dataset/sign_mnist_test/sign_mnist_test.csv")
print("Shape of train_data: ",train_data.shape,"Shape of test_data: ",test_data.shape)


In [ ]:
strlabels=list(string.ascii_uppercase) # Appending alphabet to a list.
strlabels.remove("J") # Removing J letter because of the hand gesture problem.
print(strlabels) # Making a new list that contains letters for each indices.

In [ ]:
train_labels=train_data["label"]
train_images=train_data.drop("label",axis=1).values # Dropping labels axis=1 > all column (axis=0 is all row)
test_labels=test_data["label"]
test_images=test_data.drop("label",axis=1).values # Dropping Labels axis=1 > all column (axis=0 is all row)
                                               
train_images=train_images/255.0 # We are scaling our pixels between 0 and 1 for the sake of computing performance.
test_images=test_images/255.0

train_images=train_images.reshape(-1,28,28,1) # Reshaping for making images ready to go.
test_images=test_images.reshape(-1,28,28,1)
print(train_images.shape) # We have 27.455 images as 28x28x1 (2D Image with one channel)

# **Load of 8 examples**

In [ ]:
plt.figure() # Matplotlib for visualization.
f, graph = plt.subplots(2,4) # Making space for 2 rows and 4 images for each row.
f.set_size_inches(14, 5) # Size of images.
z=0
for i in range(2): # i for rows
    for k in range(4): # k for columns
        graph[i][k].imshow(train_images[z].reshape(28,28),cmap="gray") # Showing each train image.
        graph[i][k].grid(False) # Removing grids for each train image.
        z+=1
plt.tight_layout() # Wide space for images.
plt.show()

# **Building the CNN**

### **Building the CNN**

In [ ]:
model=tf.keras.Sequential([ # Using tf.keras for build our sequential.
    tf.keras.layers.Conv2D(64,(3,3),padding="same",activation="relu",input_shape=(28,28,1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D((2,2),strides=2),
    
    tf.keras.layers.Dropout(0.2), # Dropping random 2% data out for learning variety.
    
    tf.keras.layers.Conv2D(32,(3,3),padding="same",activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D((2,2),strides=2),
    
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(16,(3,3),padding="same",activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D((2,2)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(25,activation="softmax")
])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002),loss="sparse_categorical_crossentropy",metrics=["acc"])
model.summary() # Summary of our model.

### **Data Augmentation**

In [ ]:
generator = tf.keras.preprocessing.image.ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images
generator.fit(train_images)
# We use ImageDataGenerator for changing images randomly and train our model better.

### **Callbacks**

In [ ]:
acc_treshold = 1.0 # Accuracy treshold.
class myCallback(tf.keras.callbacks.Callback):
    epc=0
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc') >= acc_treshold-0.005 and logs.get('val_acc') >= acc_treshold):
            print("\nReached %2.2f%% accuracy !" %(logs.get('val_acc')*100)) # Printing accuracy as percentage.
            self.model.stop_training = True
            self.epc=epoch+1
callbacks = myCallback()
# We are using callbacks because we want to shut our training process down when it learns 100%.

# **Training the CNN**

In [ ]:
#Hyperparameters
epochs=80
batch_size=128


history=model.fit(generator.flow(train_images,train_labels,batch_size),epochs=epochs,callbacks=[callbacks],validation_data=(test_images,test_labels),verbose=1)
# generator.flow() for applying data augmentation.
test_loss,test_acc=model.evaluate(test_images,test_labels)
print("test acc:",test_acc)

epoch_range=range(1,callbacks.epc+1 if callbacks.epc != 0 else epochs) # Epoch range for plotting our x-axis.
plt.figure() # Matplotlib for visualization.
f, ax = plt.subplots(1,2) # Making space for 2 rows and 4 images for each row.
fig.set_size_inches(30, 5) # Size of images.
plt.tight_layout() # Wide space for images.
ax[0].plot(epoch_range,history.history["acc"],color="blue",marker="o")
ax[0].plot(epoch_range,history.history["val_acc"],color="orange",marker="o")
ax[0].legend(['Training Accuracy', 'Validation Accuracy'], loc='lower right')
ax[1].plot(epoch_range,np.array(history.history["loss"]),color="blue",marker="o")
ax[1].plot(epoch_range,np.array(history.history["val_loss"]),color="orange",marker="o")
ax[1].legend(['Training Loss', 'Validation Loss'], loc='upper right')
plt.show()

# **Inference of the CNN**

Here we can test our CNN model with a random image from the test set. We can see that the model is able to predict the correct label for the image. We can also see the confidence score for the prediction. The confidence score is the probability of the image belonging to the predicted class. In this case, the confidence score is 1.0, which means that the model is 100% confident that the image belongs to the predicted class. This is because the model is trained on a small dataset and the image is from the test set.

In [ ]:
from numpy import random
n = int(random.randint(1,25))
plt.imshow(test_images[n].reshape(28,28),cmap="gray") # Showing images
plt.grid(False) # Removing grid for each image.
print("Predicted:",predictions[n],"\nLabel:",test_labels_string[n]) # Prediction - True Answer